In [48]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as snb
from datetime import datetime
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import time
from tensorflow.keras import backend as K
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input
K.clear_session()
# !pip install tensorflow

In [49]:
gbp_data_h1 = pd.read_csv('GBPUSD_H1.csv')
gbp_data_h4 = pd.read_csv('GBPUSD_H4.csv')
gbp_data_m15 = pd.read_csv('GBPUSD_M15.csv')

In [50]:
gbp_data_m15

time     open     high      low    close
0      10/18/2021 3:00  1.37542  1.37542  1.37397  1.37433
1      10/18/2021 3:15  1.37466  1.37493  1.37404  1.37422
2      10/18/2021 3:30  1.37422  1.37550  1.37419  1.37496
3      10/18/2021 3:45  1.37496  1.37571  1.37476  1.37484
4      10/18/2021 4:00  1.37484  1.37613  1.37468  1.37573
...                ...      ...      ...      ...      ...
62219  4/16/2024 13:15  1.24347  1.24451  1.24339  1.24435
62220  4/16/2024 13:30  1.24437  1.24437  1.24354  1.24382
62221  4/16/2024 13:45  1.24383  1.24402  1.24331  1.24380
62222  4/16/2024 14:00  1.24382  1.24413  1.24348  1.24370
62223  4/16/2024 14:15  1.24369  1.24454  1.24355  1.24453

[62224 rows x 5 columns]

In [51]:
gbp_data_h1.shape


(24883, 5)

In [52]:
gbp_data_h1.ndim

2

In [53]:
gbp_data_h1.size

124415

In [54]:
columns = gbp_data_h1.columns

for i in columns:
    print(gbp_data_h1[gbp_data_h1.time==None][i].count(),gbp_data_h1[i].dtype)
    

0 object
0 float64
0 float64
0 float64
0 float64


In [55]:
# gbp_data_h1["time"] = gbp_data_h1["time"].astype(datetime)

In [56]:
class FileDataset(tf.data.Dataset):
    def read_files_in_batches(num_samples):
        time.sleep(0.03)
        for sample_idx in range(num_samples):
            time.sleep(0.015)
            yield (sample_idx,)
    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls.read_files_in_batches,
            output_signature=tf.TensorSpec(shape=(1,), dtype=tf.int64),
            args=(num_samples,)
        )

In [57]:
def benchmark(dataset, num_epochs=2):
    for epoch_num in range(num_epochs):
        for sample in dataset:
            time.sleep(0.01)

In [58]:
%%timeit
benchmark(FileDataset())

2024-04-27 15:20:47.521723: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-27 15:20:47.649232: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-27 15:20:47.784797: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-27 15:20:47.908109: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-27 15:20:48.051991: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-27 15:20:48.169287: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-27 15:20:48.317771: W tensorflow/core/framework/local_rendezvous.cc:404] L

257 ms ± 6.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


2024-04-27 15:20:49.336502: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-27 15:20:49.452861: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [59]:
X = gbp_data_h1[['open', 'high', 'low']].values
y = gbp_data_h1['close'].values


In [60]:
label_encoder = LabelEncoder()

In [61]:
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [65]:
model = Sequential([
    Input(shape=(X_train.shape)),
    LSTM(50),
    Dense(1)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [66]:
X_train.shape[1]

3

In [67]:
X_train.size
X_train.shape

(19906, 3)

In [69]:
# X_train_reshaped = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
# X_test_reshaped = X_test.reshape(X_train.shape[0], 1, X_test.shape[1])

In [70]:
# y_train_encoded = label_encoder.fit_transform(y_train)
# y_test_encoded = label_encoder.fit_transform(y_test)

In [71]:
X_train.shape
X_test.shape
y_train.shape
y_test.shape
X_train_reshaped.shape
X_test_reshaped.shape 

(4977, 1, 3)

In [72]:
print(X_train.shape)
print(X_test.shape)

(19906, 3)
(4977, 3)


In [73]:
# history = model.fit( X_train_reshaped, y_train_encoded, epochs=50, batch_size=32, validation_data=( X_test_reshaped, y_test_encoded))

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 3), dtype=float32). Expected shape (None, 19906, 3), but input has incompatible shape (None, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 3), dtype=float32)
  • training=True
  • mask=None

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

predictions = model.predict(X_test)